# And yet again about functions

## Function introspection

Ранее мы рассматривали простой пример функции, у которой задана док-строка. Эту док-строку мы могли получить с помощью вызова `help` или напрямую через поле `__doc__`.

Вспомним:

In [8]:
def func(name):  # name and function arguments
    '''
    Doc string to be shown in help
    '''
    print(f"Hello, world! {name} is greeting you!") # body

func('Denis')

Hello, world! Denis is greeting you!


In [9]:
func.__doc__

'\n    Doc string to be shown in help\n    '

Но `__doc__` -- это далеко не единственное "скрытое" поле у функции

In [10]:
dir(func)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

Какие-то из этих атрибутов общие для всех объектов в Питоне (а мы помним, что функция -- это тоже объект, как, например, и лист)

Интересно, но у функции, как и других объектов, есть `__dict__`

In [14]:
func.__dict__

{}

In [18]:
func.alpha = 0
func.__dict__

{'alpha': 0}

In [21]:
func.__dict__.clear()

На практике редко используют атрибуты для функций. Но вот такой пример есть в Django:

```python
def upper_case_name(obj):
    return ("%s %s" % (obj.first_name, obj.last_name)).upper()
upper_case_name.short_description = 'Customer name'
```

Но все-таки рассмотрим сначала методы, характерные именно для функций

In [75]:
class A: pass
ob = A()

func_magics = set(dir(func))
obj_magics = set(dir(A))

sorted(func_magics - obj_magics)

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

## Function parameters

In [44]:
def create_tag(name, *content, class_name=None, **attrs):
    '''
    Generate html tag
    '''
    if class_name is not None:
        attrs['class'] = class_name
    
    attr_str = ''
    if attrs:
        attr_str = ' ' + ' '.join([f'{name}="{value}"' for name, value in attrs.items()])
    
    if content:
        return '\n'.join([f'<{name}{attr_str}> {item} </{name}>' for item in content])
    else:
        return f'<{name}{attr_str} />'

Давайте еще раз пройдемся по тому, как можно вызвать функцию с параметрами

1. Передаем только `name` как позиционный аргумент

In [45]:
create_tag('p')

'<p />'

2. Передаем `name` как первый позиционный, остальные позиция подхватываются через `*content`

In [58]:
create_tag('p', 'hello')

'<p> hello </p>'

In [50]:
print(create_tag('p', 'hello', 'world'))

<p> hello </p>
<p> world </p>


3. После `name` и `*content` ожидаются keyword arguments. Один указан явно, другие подхватятся через `**attrs`

In [59]:
print(create_tag('p', 'hello', 'world', id=42))

<p id="42"> hello </p>
<p id="42"> world </p>


In [60]:
print(create_tag('p', 'hello', 'world', class_name='hw', id=42))

<p id="42" class="hw"> hello </p>
<p id="42" class="hw"> world </p>


In [64]:
print(create_tag('foo', 'hello', 'world', **{'id': 42, 'creator': 'me'}))

<foo id="42" creator="me"> hello </foo>
<foo id="42" creator="me"> world </foo>


4. Первый позиционный аргумент тоже может быть передан через keyword

In [63]:
print(create_tag(content='test', name='bar'))

<bar content="test" />


5. Мы можем задать весь тег как словарь

In [65]:
predefined_tag = {
    'name' : 'p',
    'title': 'Hello, World!',
    'src': 'Sun',
    'class_name': 'hw'
}

In [66]:
print(create_tag(**predefined_tag))

<p title="Hello, World!" src="Sun" class="hw" />


In [67]:
predefined_tag = {
    'name' : 'p',
    'content': ['this', 'is', 'good'],
    'title': 'Hello, World!',
    'src': 'Sun',
    'class_name': 'hw'
}

In [68]:
print(create_tag(**predefined_tag))

<p content="['this', 'is', 'good']" title="Hello, World!" src="Sun" class="hw" />


## Functions magics

### Function args

In [82]:
def test_str(s, max_len=10, name='test'):
    some_var = 0
    other_var = '10'
    return s[:max_len]

In [84]:
test_str.__defaults__

(10, 'test')

In [86]:
test_str.__code__

<code object test_str at 0x7f6c733d5ed0, file "/tmp/ipykernel_722055/2346920870.py", line 1>

In [87]:
test_str.__code__.co_varnames

('s', 'max_len', 'name', 'some_var', 'other_var')

In [88]:
test_str.__code__.co_argcount

3

### Annotations

В Python можно назначить метаданные для аргументов и выходного значения функции 

In [89]:
def test_str(s: str, max_len: int = 10, name: str = 'test') -> int:
    some_var = 0
    other_var = '10'
    return s[:max_len]

Аннотации могут быть любым выражением. Они игнорируются во время выполнения программы. Доступ можно получить через поле `__annotations__`

In [90]:
test_str.__annotations__

{'s': str, 'max_len': int, 'name': str, 'return': int}

Эту информацию могут использовать декораторы, ide или библиотеки (например, для проверки синтаксиса)

In [91]:
from inspect import signature

In [92]:
sig = signature(test_str)

In [93]:
sig.return_annotation

int

In [95]:
sig.parameters.values()

odict_values([<Parameter "s: str">, <Parameter "max_len: int = 10">, <Parameter "name: str = 'test'">])

## Elements of functional programming

Немного об этом от самого [Гвидо](http://python-history.blogspot.com/2009/04/origins-of-pythons-functional-features.html)

In [116]:
from operator import itemgetter, attrgetter, methodcaller

**itemgetter**

In [103]:
some_data = [
    ('A', 'JP', 36.933, (1, 139.691667)),
    ('B', 'IN', 21.935, (28.613889, 77.208889)),
    ('C', 'MX', 20.142, (19.433333, -99.133333)),
    ('A', 'US', 20.104, (-1, -74.020386)),
    ('B', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [104]:
for city in sorted(some_data, key=itemgetter(0)):
    print(city)

('A', 'JP', 36.933, (1, 139.691667))
('A', 'US', 20.104, (-1, -74.020386))
('B', 'IN', 21.935, (28.613889, 77.208889))
('B', 'BR', 19.649, (-23.547778, -46.635833))
('C', 'MX', 20.142, (19.433333, -99.133333))


In [105]:
for city in sorted(some_data, key=itemgetter(0, 3)):
    print(city)

('A', 'US', 20.104, (-1, -74.020386))
('A', 'JP', 36.933, (1, 139.691667))
('B', 'BR', 19.649, (-23.547778, -46.635833))
('B', 'IN', 21.935, (28.613889, 77.208889))
('C', 'MX', 20.142, (19.433333, -99.133333))


Получили сортировку по нескольким параметрам. Также можно написать и с помощью лямбды

In [107]:
name = itemgetter(0, 1)  # uses __getitem__

for city in some_data:
    print(name(city))

('A', 'JP')
('B', 'IN')
('C', 'MX')
('A', 'US')
('B', 'BR')


**attrgetter**

In [109]:
from collections import namedtuple

In [111]:
Subject = namedtuple('Subject', 'name difficulty')

In [112]:
python_course = Subject('Python', 1)
python_course

Subject(name='Python', difficulty=1)

In [114]:
sub_name = attrgetter('name')

In [115]:
sub_name(python_course)

'Python'

**methodcaller**

In [117]:
sample_str = 'This is really functional'

In [118]:
upcase = methodcaller('upper')
upcase(sample_str)

'THIS IS REALLY FUNCTIONAL'

In [120]:
repl = methodcaller('replace', ' ', '!')
repl(sample_str)

'This!is!really!functional'

И еще много подобных методов

**Functools.partial**

Иногда бывает, что в конкретной ситуации вы хотите вызывать функцию с фиксированным конкретным параметром, у которого нет дефолтного значения

In [123]:
from operator import mul
from functools import partial

In [126]:
mul(4, 3)

12

In [127]:
quadriple = partial(mul, 4)

In [128]:
quadriple(3)

12

In [130]:
list(map(quadriple, range(10)))

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36]

Мы не могли бы использовать `mul` в `map` без такой модификации

Callable objects with class example

## Decorators

На прошлом семинаре мы уже затрагивали декораторы. Давайте рассмотрим их поглубже.

Декоратор -- это callable, который принимает на вход функцию в качестве аргумента ("декорирует" ее)

### Recap

Пример, который мы рассматривали:

In [132]:
def my_decorator(func):
    def decorated(a, b):  # название можно менять
        print("doing smth before")
        func(a, b)
        print("doing smth after")
    return decorated

def func(a, b):
    print(a + b)

@my_decorator
def new_func(a, b):
    print(a + b)
    
func = my_decorator(func)

In [133]:
func(2, 3)

doing smth before
5
doing smth after


In [134]:
new_func(2, 3)

doing smth before
5
doing smth after


### Замена исходной функции

Функция, возвращаемая декоратором, заменяет декорируемую

In [136]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [139]:
def target():
    print('running target()')

In [140]:
@deco
def target():
    print('running target()')

In [141]:
target()

running inner()


In [143]:
target  # now reference to inner 

<function __main__.deco.<locals>.inner()>

### Running at import time

Key feature декораторов в том, что они применяются сразу после объявления декорируемой функции во время загрузки модуля в Python 

In [147]:
registry = []

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def run():
    print('running run()')
    print('registry', registry)
    f1()
    f2()
    f3()

running register <function f1 at 0x7f6c72626f80>
running register <function f2 at 0x7f6c72626ef0>


In [148]:
run()

running run()
registry [<function f1 at 0x7f6c72626f80>, <function f2 at 0x7f6c72626ef0>]
running f1()
running f2()
running f3()


Поинт в том, что декораторы применяются при импорте модуля, но сами декорируемые функции отрабатывают только при непосредственном их вызове

### Variable scope

In [160]:
from dis import dis

In [164]:
def func_1(a):
    print(a)
    print(b)
    
func_1(42)

42


NameError: name 'b' is not defined

In [165]:
dis(func_1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


Логичная ошибка

In [168]:
b = 1337

func_1(42)

42
1337


А что если?

In [169]:
b = 6
def func_2(a):
    print(a)
    print(b)
    b = 9
    
func_2(42)

42


UnboundLocalError: local variable 'b' referenced before assignment

Посмотрим в [dis](https://docs.python.org/3/library/dis.html)

In [172]:
dis(func_1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [170]:
dis(func_2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


Заметим, что 42 у нас напечаталось! Почему не напечаталось b?

Потому что в Python предполагается, что локальные переменные объявлены внутри функции. И поскольку объявление b внутри функции есть, мы получаем ошибку.

Как можно поправить?

In [157]:
b = 6
def func_fixed(a):
    global b
    print(a)
    print(b)
    b = 9
    
func_fixed(42)

42
6


Но be careful!

In [154]:
b

9

### Closures

Замыкания -- это функции, которые при своей работе используют переменные, не объявленные внутри самой функции и при этом не являющиеся глобальными. 

Сложно

### naive example

Разберем на примерах

Пусть есть функция `average`, которая считает среднее. В эту функцию при этом постоянно подаются новые данные, и она должна считать среднее с учетом предыдущих обработанных значений.

```python
    average(2)  # 2
    average(6)  # (6 + 2) / 2 = 4
    average(7)  # (6 + 2 + 8) / 3 = 5
```

In [175]:
def make_averager():
    history = []
    
    def averager(new_val):
        history.append(new_val)
        total = sum(history)
        return total / len(history)
    
    return averager

In [176]:
average = make_averager()

In [177]:
average(2)

2.0

In [178]:
average(6)

4.0

In [179]:
average(7)

5.0

In [180]:
average.__code__.co_varnames

('new_val', 'total')

Переменная `history` в данном контексте называется свободной

In [181]:
average.__code__.co_freevars

('history',)

In [182]:
average.__closure__

(<cell at 0x7f6c7282a4d0: list object at 0x7f6c72c60eb0>,)

In [183]:
average.__closure__[0].cell_contents

[2, 6, 7]

To summarize: a closure is function that retains the bindings of the free variables that
exist when the function is defined, so that they can be used later when the function is
invoked and the defining scope is no longer available.

### effective example

В чем проблема имплементации выше? Как можно ее улучшить?

In [197]:
def make_averager():
    avg_cnt = 0
    running_total = 0
    
    def averager(new_val):
        avg_cnt += 1
        running_total += new_val
        return running_total / avg_cnt
    
    return averager

In [198]:
average = make_averager()

In [186]:
average(2)

UnboundLocalError: local variable 'avg_cnt' referenced before assignment

Пример выше не работает потому что числовые типы у нас неизменяемые. И когда во внутренней функции происходит вызов `+=`, мы формируем новую локальную переменную, которая уже не имеет отношения к исходной.

In [199]:
average.__code__.co_varnames, average.__code__.co_freevars

(('new_val', 'avg_cnt', 'running_total'), ())

Простой способ это пофиксить -- завернуть во что-то, поддерживающее inplace изменение, например, dict

In [200]:
def make_averager_fixed_naive():
    stats_dict = {
        'avg_cnt': 0,
        'running_total': 0
    }
    
    def averager(new_val):
        stats_dict['avg_cnt'] += 1
        stats_dict['running_total'] += new_val
        return stats_dict['running_total'] / stats_dict['avg_cnt']
    
    return averager

In [201]:
average = make_averager_fixed_naive()

In [189]:
average(2)

2.0

In [190]:
average(6)

4.0

In [191]:
average(7)

5.0

In [202]:
average.__code__.co_varnames, average.__code__.co_freevars

(('new_val',), ('stats_dict',))

Лучший способ -- использование ключевого слова `nonlocal`, которое говорит, что переданные переменные нужно воспринимать как свободные

In [203]:
def make_averager_fixed_best():
    avg_cnt = 0
    running_total = 0
    
    def averager(new_val):
        nonlocal avg_cnt, running_total
        avg_cnt += 1
        running_total += new_val
        return running_total / avg_cnt
    
    return averager

In [204]:
average = make_averager_fixed_best()

In [205]:
average.__code__.co_varnames, average.__code__.co_freevars

(('new_val',), ('avg_cnt', 'running_total'))

In [194]:
average(2)

2.0

In [195]:
average(6)

4.0

In [196]:
average(7)

5.0

### Implementing decorator

In [214]:
import time

In [233]:
def clock(func):
    def clocked(*args):
        t_start = time.time()
        res = func(*args)
        total_time = time.time() - t_start
        print(f'{func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res
    return clocked

In [234]:
@clock
def slow_fact(n):
    time.sleep(1)
    if n == 0:
        return 1  # условие выхода из рекурсии
    return n * slow_fact(n - 1)

In [235]:
slow_fact(10)

slow_fact((0,)) -> 1 executed in 1.00s
slow_fact((1,)) -> 1 executed in 2.00s
slow_fact((2,)) -> 2 executed in 3.00s
slow_fact((3,)) -> 6 executed in 4.01s
slow_fact((4,)) -> 24 executed in 5.01s
slow_fact((5,)) -> 120 executed in 6.01s
slow_fact((6,)) -> 720 executed in 7.01s
slow_fact((7,)) -> 5040 executed in 8.01s
slow_fact((8,)) -> 40320 executed in 9.01s
slow_fact((9,)) -> 362880 executed in 10.01s
slow_fact((10,)) -> 3628800 executed in 11.01s


3628800

**Real world example**

In [236]:
from functools import lru_cache

In [242]:
@clock
def fib(n):
    time.sleep(1)
    if n < 2:
        return n
    return fib(n - 2) + fib(n - 1)

In [244]:
%%time
fib(10)

fib((0,)) -> 0 executed in 1.00s
fib((1,)) -> 1 executed in 1.00s
fib((2,)) -> 1 executed in 3.01s
fib((1,)) -> 1 executed in 1.00s
fib((0,)) -> 0 executed in 1.00s
fib((1,)) -> 1 executed in 1.00s
fib((2,)) -> 1 executed in 3.00s
fib((3,)) -> 2 executed in 5.01s
fib((4,)) -> 3 executed in 9.02s
fib((1,)) -> 1 executed in 1.00s
fib((0,)) -> 0 executed in 1.00s
fib((1,)) -> 1 executed in 1.00s
fib((2,)) -> 1 executed in 3.00s
fib((3,)) -> 2 executed in 5.01s
fib((0,)) -> 0 executed in 1.00s
fib((1,)) -> 1 executed in 1.00s
fib((2,)) -> 1 executed in 3.00s
fib((1,)) -> 1 executed in 1.00s
fib((0,)) -> 0 executed in 1.00s
fib((1,)) -> 1 executed in 1.00s
fib((2,)) -> 1 executed in 3.00s
fib((3,)) -> 2 executed in 5.01s
fib((4,)) -> 3 executed in 9.01s
fib((5,)) -> 5 executed in 15.02s
fib((6,)) -> 8 executed in 25.04s
fib((1,)) -> 1 executed in 1.00s
fib((0,)) -> 0 executed in 1.00s
fib((1,)) -> 1 executed in 1.00s
fib((2,)) -> 1 executed in 3.00s
fib((3,)) -> 2 executed in 5.01s
fib((0,)

55

In [246]:
@lru_cache()
def fib(n):
    time.sleep(1)
    if n < 2:
        return n
    return fib(n - 2) + fib(n - 1)

In [247]:
%%time
fib(10)

CPU times: user 0 ns, sys: 6.9 ms, total: 6.9 ms
Wall time: 11 s


55

### Parametrized decorator

In [252]:
def clock(active=False):
    def decorate(func):
        def clocked(*args):
            if not active:
                return func(*args)
            t_start = time.time()
            res = func(*args)
            total_time = time.time() - t_start
            print(f'{func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
            return res
        return clocked
    return decorate

In [253]:
@clock(active=False)
def slow_fact(n):
    time.sleep(1)
    if n == 0:
        return 1
    return n * slow_fact(n - 1)

In [254]:
slow_fact(3)

6

In [255]:
@clock(active=True)
def slow_fact(n):
    time.sleep(1)
    if n == 0:
        return 1
    return n * slow_fact(n - 1)

In [256]:
slow_fact(3)

slow_fact((0,)) -> 1 executed in 1.00s
slow_fact((1,)) -> 1 executed in 2.00s
slow_fact((2,)) -> 2 executed in 3.00s
slow_fact((3,)) -> 6 executed in 4.01s


6

### Bonus Task

Напишите декоратор, который может:
- поддерживать кэш из обработанных значений
- показать последние K обработанных значений (параметр)
- быть отключаемым (параметр)

С помощью такого кэша реализуйте проверку первых N чисел на простоту. Сравните время выполнения операций со включенным и выключенным кэшем.

- попробуйте реализовать кэш по логике, похожей на LRU. Продемонстрируйте его работу на примере.

Можно использовать следующую функцию

In [257]:
import time

def do_heavy_calculation(arg):
    time.sleep(2)
    return arg